In [12]:
import torch
import numpy as np
import sys
from torch.utils.data import Dataset, DataLoader


sys.path.append("..")
from model import ParticleEventTransformer
from data import get_database_path, get_h5_files, read_h5_file, select_events


device = "cuda" if torch.cuda.is_available() else "mps" if sys.platform == "darwin" else "cpu"


In [26]:
database_path = get_database_path()
bkg_files, sig_files = get_h5_files()


# SM processes
particle_type_scale = 20
bkg = read_h5_file(database_path, bkg_files[0]['file'])
# SM processes

neutral_boson = read_h5_file(database_path, sig_files[0]['file'])
# A neutral scalar boson (A) with mass 50 GeV, decaying to two off-shell Z bosons, each forced to decay to two leptons: A → 4l

leptoquark = read_h5_file(database_path, sig_files[1]['file'])
# A leptoquark (LQ) with mass 80 GeV, decaying to a b quark and a τ lepton24

neutral_higgs = read_h5_file(database_path, sig_files[2]['file'])
# A scalar boson with mass 60 GeV, decaying to two tau leptons: h0→ ττ

charged_higgs = read_h5_file(database_path, sig_files[3]['file'])
# A charged scalar boson with mass 60 GeV, decaying to a tau lepton and a neutrino

In [32]:
feature_size = 3  # pT, eta, phi
embed_size = 16  # Dimension of embedding space
num_heads = 8    # Number of attention heads
num_layers = 4   # Number of nn.TransformerEncoderLayer
hidden_dim = 256 # Dimension of the feedforward network model in nn.TransformerEncoder
output_dim = 2  # Final dimension of the output embedding


model = ParticleEventTransformer(feature_size, embed_size, num_heads, hidden_dim, output_dim, num_layers)
model.to(device)

ParticleEventTransformer(
  (particle_embedding): Linear(in_features=3, out_features=16, bias=True)
  (pos_encoder): ParticlePositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
        )
        (linear1): Linear(in_features=16, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=16, bias=True)
        (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=304, out_features=2, bias=True)
)

In [28]:
from analysis import inference

In [29]:
from data import EventDataset

In [30]:
infer_test_num = 100000
bkg_infer_test = bkg[:infer_test_num]
infer_dataset = EventDataset(bkg_infer_test)
infer_dataloader = DataLoader(infer_dataset, batch_size=256, num_workers=16, prefetch_factor=5)

In [33]:
model.load_state_dict(torch.load("checkpoint.pt"))

embed_points = inference(model, infer_dataloader, embed_dim=output_dim)

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/391 [00:01<?, ?it/s]


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float